

## Random thoughts and notes (disregard)
- Do feature importance test
- Add scaling
- Think about proper way to test (for time-seriers), that will correspond to the submission and real time testing. For example, maybe our test needs to be two months removed from the training (similar as in real-scenario)


# Loading Libraries and Datasets

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import tensorflow as tf
import os
import gc

from sklearn import set_config
from sklearn.base import clone
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')

pd.set_option('display.max_rows', 100)
#set_config(transform_output = 'pandas')
pd.options.mode.chained_assignment = None

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Due to the size of dataset, we can load all integral features into bytes (int 8) and short (int 16). We can also try to load float features into float 32 or float 16 if we want. You have to keep in mind that this sacrifices the accuracy of our calculation though.

In [26]:
dtypes = {
    'stock_id' : np.uint8,
    'date_id' : np.uint16,
    'seconds_in_bucket' : np.uint16,
    'imbalance_buy_sell_flag' : np.int8,
    'time_id' : np.uint16,
}
path = "C:\\Users\\lukab\\Desktop\\Optiver\\"
train = pd.read_csv(path + 'train.csv', dtype = dtypes).drop(['row_id'], axis = 1)
test = pd.read_csv('test.csv', dtype = dtypes).drop(['row_id'], axis = 1)

gc.collect()

4228

## 1) Using the entire dataset

In [27]:
train = train[0:50000]
X = train[train.columns[2:-2]]
# I choose to fillna with 1
X = X.fillna(1)

In [28]:
y = train[train.columns[-2]]
y

0        -3.029704
1        -5.519986
2        -8.389950
3        -4.010200
4        -7.349849
           ...    
49995     2.759695
49996    15.419722
49997    -0.330210
49998     1.189709
49999     3.449917
Name: target, Length: 50000, dtype: float64

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

names = [
         "KNN Regression"
        ]

models = [
   KNeighborsRegressor(),
]

parameters = [{
              'clf__n_neighbors': [i for i in range(1,31)],
              'clf__weights':['uniform', 'distance'],
              'clf__algorithm':['ball_tree','kd_tree']
}
             ]

count=0
for name, model, params in zip(names, models, parameters):
    count += 1
    clf_pipe = Pipeline([
        ('clf',model)])
    gs_clf = GridSearchCV(clf_pipe, param_grid=params, n_jobs=-1,scoring='neg_mean_absolute_error',verbose=10)
    clf = gs_clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    final_score = np.round(mean_absolute_error(y_test, y_pred),3)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


In [30]:
# Gridsearch results
results_df = pd.concat([pd.DataFrame(clf.cv_results_["params"]),pd.DataFrame(clf.cv_results_["mean_test_score"], columns=["mean_test_score"])],axis=1)
results_df = results_df.sort_values(by=["mean_test_score"],ascending= False)
results_df['mean_absolute_error'] = -1 * results_df["mean_test_score"].round(3)
results_df.drop('mean_test_score',axis=1,inplace=True)
results_df.head(10)

,clf__algorithm,clf__n_neighbors,clf__weights,mean_absolute_error
13,ball_tree,7,distance,5.109
73,kd_tree,7,distance,5.109
77,kd_tree,9,distance,5.112
17,ball_tree,9,distance,5.112
75,kd_tree,8,distance,5.112
15,ball_tree,8,distance,5.112
25,ball_tree,13,distance,5.113
85,kd_tree,13,distance,5.113
87,kd_tree,14,distance,5.114
27,ball_tree,14,distance,5.114


In [31]:
print('The final score is',final_score,'with the model',name,'with parameters',clf.best_params_, 'that had a cross-validation score of',abs(np.round(clf.best_score_,3)),'.')

The final score is 4.83 with the model KNN Regression with parameters {'clf__algorithm': 'ball_tree', 'clf__n_neighbors': 7, 'clf__weights': 'distance'} that had a cross-validation score of 5.109 .


In [1]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))
    sample_prediction['target'] = 0
    env.predict(sample_prediction)
    counter += 1

ModuleNotFoundError: No module named 'optiver2023'